In [10]:
import pandas as pd
import numpy as np

''

# Direct SCF with different xc

In [12]:
A100_file = 'A100-SXM-80GB.csv'
qchem_file = 'qchem-32-cores-cpu.csv'

keys = ['mol', 'natm']
empty = {'mol':[], 'natm':[]}
df_A100_scf = pd.DataFrame(empty)
df_V100_scf = pd.DataFrame(empty)
df_A100_grad = pd.DataFrame(empty)
df_V100_grad = pd.DataFrame(empty)
path = 'water_clusters/xc/'
for xc in ['LDA', 'PBE', 'B3LYP', 'M06', 'wB97m-v']:
    df_qchem = pd.read_csv(path + xc + '/' + qchem_file)
    df_qchem = df_qchem.rename(columns={'t_scf':'scf_qchem', 't_gradient':'grad_qchem'})
    
    df_A100 = pd.read_csv(path + xc + '/' + A100_file)
    df_A100 = df_A100.rename(columns={'t_scf':'scf_A100', 't_gradient':'grad_A100'})
    df_A100 = df_A100.merge(df_qchem, how='outer', on='mol')
    
    df_A100['scf_'+xc] = df_A100['scf_qchem']/df_A100['scf_A100']
    df_A100['grad_'+xc] = df_A100['grad_qchem']/df_A100['grad_A100']
    df_A100 = df_A100[keys+['scf_'+xc, 'grad_'+xc]]
    
    df_A100_scf = df_A100_scf.merge(df_A100[keys+['scf_'+xc]], how='outer', on=keys)
    df_A100_grad= df_A100_grad.merge(df_A100[keys+['grad_'+xc]], how='outer', on=keys)
    df_A100_scf = df_A100_scf.rename(columns={'scf_'+xc:xc})
    df_A100_grad = df_A100_grad.rename(columns={'grad_'+xc:xc})
    df_A100_scf[xc] = df_A100_scf[xc].apply(lambda x: round(x,2))
    df_A100_grad[xc] = df_A100_grad[xc].apply(lambda x: round(x,2))

print(df_A100_scf.to_markdown(index=False))

''

|   mol |   natm |    LDA |    PBE |   B3LYP |    M06 |   wB97m-v |
|------:|-------:|-------:|-------:|--------:|-------:|----------:|
|     2 |      3 |   0.22 |   0.32 |    0.27 |   0.25 |      0.69 |
|     3 |     15 |   0.68 |   0.25 |    1.58 |   2.61 |      4.84 |
|     4 |     30 |   1.59 |   2.63 |    4.09 |   6.93 |      8.17 |
|     5 |     60 |   2.86 |   3.64 |    7.15 |   8.44 |      9.44 |
|     6 |     96 |   4.34 |   4.39 |    7.75 |  10.58 |      9.87 |
|     7 |    141 |   4.07 |   4.1  |    8.87 |  10.47 |     10.13 |
|     8 |    228 |   4.34 |   4.58 |    9.39 |  10.48 |      9.36 |
|     9 |    300 |   5.05 |   5.21 |    9.35 |  11.36 |    nan    |
|    10 |    417 |   4.91 | nan    |  nan    | nan    |    nan    |
|    10 |    nan | nan    | nan    |  nan    | nan    |    nan    |


In [14]:
print(df_A100_grad.to_markdown(index=False))

''

|   mol |   natm |    LDA |    PBE |   B3LYP |    M06 |   wB97m-v |
|------:|-------:|-------:|-------:|--------:|-------:|----------:|
|     2 |      3 |   0.82 |   0.89 |    0.75 |   0.82 |      0.6  |
|     3 |     15 |   0.39 |   0.19 |    1.46 |   1.52 |      1.47 |
|     4 |     30 |   0.56 |   1.04 |    2.07 |   2.25 |      1.89 |
|     5 |     60 |   0.54 |   0.87 |    2.42 |   2.4  |      1.77 |
|     6 |     96 |   0.6  |   0.87 |    2.36 |   2.51 |      1.53 |
|     7 |    141 |   0.93 |   1.1  |    2.61 |   2.59 |      1.55 |
|     8 |    228 |   1.92 |   1.9  |    3.37 |   3.39 |      1.83 |
|     9 |    300 |   2.26 |   2.02 |    3.06 |   3.59 |    nan    |
|    10 |    417 |   2.46 | nan    |  nan    | nan    |    nan    |
|    10 |    nan | nan    | nan    |  nan    | nan    |    nan    |
